In [1]:
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt


In [10]:
ticker = 'INFY'
print(ticker)
ticker = ticker + '.NS'
a1 = yf.Ticker(ticker)
df = a1.history(period="200d" , interval = '1d')

# high_1 = 1000
# low_1 = -1000
data = df
position=0
position_index=0
target= 10
stoploss= 10
realised_pnl=0
number_of_trades=0
number_of_trades_pr=0
pnl_l = 0
pnl_h = 0
pnl_arr = []
position_l = []
tradebook = {}
profit = []
loss = []
nav = []
pnl_l1 =[]

INFY


In [11]:
def BollingerBands (df0, period, dev):
    df = df0
    sma = df["Close"].rolling(window = period).mean()
    rstd = df["Close"].rolling(window = period).std()
    df["Standard Moving Average"] = sma
    df["std"] = rstd
    df["Upper Band"] = sma + dev * rstd
    df["Lower Band"] = sma - dev * rstd

    return df

In [13]:
realdata = BollingerBands(data , 20 , 1)
for i in range(len(data)):
    if((position == 0) & (realdata["Close"][i] > realdata["Upper Band"][i])):
        position_date=data.index[i].date()
        position_index=data.index[i]
        entry_price = data.loc[data.index[i+1],'Open']
        position = 1
        print('long position on {}'.format(data.index[i]))
        number_of_trades=number_of_trades+1
        position_l.append(position_index)
        continue


    if((position == 0) & (realdata["Close"][i] < realdata["Lower Band"][i])):
        position_date=data.index[i].date()
        position_index=data.index[i]
        entry_price = data.loc[data.index[i+1],'Open']
        position = -1
        print('short position on {}'.format(data.index[i]))
        number_of_trades=number_of_trades+1
        position_l.append(position_index)
        continue

    if (position_index!=0):
        pnl = ((data.loc[data.index[i],'Close']- entry_price)/entry_price)*(position)*100
        pnl_amt = data.loc[data.index[i],'Close']- entry_price*(position)*50

    if ((position != 0) and ((pnl>target) or (pnl < -stoploss))):
        if(pnl>target):
            number_of_trades_pr += 1
            profit.append(pnl_amt)
        if(pnl>target):
            loss.append(pnl_amt)
        realised_pnl=realised_pnl+pnl_amt
        pnl_l1.append(pnl_amt)
        position=0
        print('position exited at {}'.format(data.index[i].date()))
        position_index=0
        tradebook[number_of_trades] = {'entry_date' :position_date ,'entry_price': entry_price ,'long_short': position ,'exit_date' :data.index[i], 'pnl' :pnl}
        currval = 100000 + pnl_amt
        nav.append(currval)
        print(f"{pnl}%")
        print(f"{pnl_amt}%")


print(f"Net PnL is Rs {realised_pnl}")
print(f"Max Profit is Rs {max(pnl_l1)}")
print(f"Max Loss is Rs {-min(pnl_l1)}")
print(f"ROI is Rs {pnl_amt/100000}")
print(f"Hit Rate is {number_of_trades_pr/number_of_trades}")
print(f"Reward to risk ratio is {np.mean(profit)/np.mean(loss)}")
minnav = min(nav)
maxdd = ((100000 - minnav)/100000)*100
print(f"Max drawdown is {maxdd}%")

position exited at 2023-04-17
15.365730782798565%
72662.48913282054%
short position on 2023-04-18 00:00:00+05:30
position exited at 2023-07-04
-10.142295688226003%
61611.86133378028%
long position on 2023-07-05 00:00:00+05:30
position exited at 2023-09-05
10.156050164234733%
-64809.73214101077%
long position on 2023-09-06 00:00:00+05:30
Net PnL is Rs 142127.1074584106
Max Profit is Rs 72662.48913282054
Max Loss is Rs 64809.73214101077
ROI is Rs -0.7115732001268775
Hit Rate is 0.75
Reward to risk ratio is 1.0
Max drawdown is 64.80973214101077%


In [14]:
df2 = a1.history(period="500d" , interval = '1d')
data = df2
position=0
position_index=0
target= 10
stoploss= 10
realised_pnl=0
number_of_trades=0
number_of_trades_pr=0
pnl_l = 0
pnl_h = 0
pnl_arr = []
position_l = []
tradebook = {}
profit = []
loss = []
nav = []

In [15]:
def RSI(df0, period):
    df = df0
    df["delta"] = df["Close"].diff()
    up, down = df["delta"].copy(), df["delta"].copy()
    up[up<0] = 0
    down[down>0] = 0
    df["up"] = up
    df["down"] = down
    avgain_sma = df["up"].rolling(window = period).mean()
    avloss_sma = df["down"].abs().rolling(window = period).mean()
    avgain_exp = df["up"].ewm(span=period).mean()
    avloss_exp = df["down"].abs().ewm(span=period).mean()
    avgain_smoothened = ((df["up"].ewm(span=period).mean()* period - df['up']) * 13) + (df['up'])
    avloss_smoothened = ((df["down"].abs().ewm(span=period).mean()* period - df['down'].abs()) * 13) + (df['down'].abs())
    rs_sma = avgain_sma/avloss_sma
    rs_exp = avgain_exp/avloss_exp
    rs_smoothened = avgain_smoothened/avloss_smoothened

    rsi_exp = 100.0 - (100.0/(1.0 + rs_exp))
    rsi_sma = 100.0 - (100.0/(1.0 + rs_sma))
    rsi_smoothened = 100.0 - (100.0/(1.0 + rs_smoothened))

    df["SMA RSI"] = rsi_sma
    df["EMA RSI"] = rsi_exp
    df["Smoothened RSI"] = rsi_smoothened
    return df
realdata = RSI(data, 27)

In [16]:
for i in range (len(realdata)):
    if((position == 0) & (realdata["Smoothened RSI"][i] < 20)):
        position_date=data.index[i].date()
        position_index=data.index[i]
        entry_price = data.loc[data.index[i+1],'Open']
        position = 1
        print('long position on {}'.format(data.index[i]))
        number_of_trades=number_of_trades+1
        position_l.append(position_index)
        continue

    if((position == 0) & (realdata["Smoothened RSI"][i] > 80)):
        position_date=data.index[i].date()
        position_index=data.index[i]
        entry_price = data.loc[data.index[i+1],'Open']
        position = -1
        print('short position on {}'.format(data.index[i]))
        number_of_trades=number_of_trades+1
        position_l.append(position_index)
        continue

    if (position_index!=0):
        pnl = ((data.loc[data.index[i],'Close']- entry_price)/entry_price)*(position)*100
        pnl_amt = (data.loc[data.index[i],'Close']- entry_price)*position*50

    if ((position != 0) and ((pnl>target) or (pnl < -stoploss))):

        if(pnl>target):
            number_of_trades_pr += 1
            profit.append(pnl_amt)
        if(pnl>target):
            loss.append(pnl_amt)
        realised_pnl=realised_pnl+pnl_amt
        pnl_l1.append(pnl_amt)
        position=0
        print('position exited at {}'.format(data.index[i].date()))
        position_index=0
        tradebook[number_of_trades] = {'entry_date' :position_date ,'entry_price': entry_price ,'long_short': position ,'exit_date' :data.index[i], 'pnl' :pnl}
        currval = 100000 + pnl_amt
        nav.append(currval)
        print(f"{pnl}%")
        print(f"{pnl_amt}%")

print(f"Net PnL is Rs {realised_pnl}")
print(f"Max Profit is Rs {max(pnl_l1)}")
print(f"Max Loss is Rs {-min(pnl_l1)}")
print(f"ROI is Rs {pnl_amt/100000}")
print(f"Hit Rate is {number_of_trades_pr/number_of_trades}")
print(f"Reward to risk ratio is {np.mean(profit)/np.mean(loss)}")
minnav = min(nav)
maxdd = ((100000 - minnav)/100000)*100
print(f"Max drawdown is {maxdd}%")

long position on 2021-11-24 00:00:00+05:30
position exited at 2021-12-28
11.058821420490201%
8843.918125513903%
Net PnL is Rs 8843.918125513903
Max Profit is Rs 72662.48913282054
Max Loss is Rs 64809.73214101077
ROI is Rs 0.08843918125513903
Hit Rate is 1.0
Reward to risk ratio is 1.0
Max drawdown is -8.843918125513897%
